In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
spending_features = [
    'MntWines', 'MntFruits', 'MntMeatProducts',
    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'Total_Spending'
]





2. **Data Preparation**: We will standardize the selected features to ensure that they are on the same scale.

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load dataset
df = pd.read_csv('featured_customer_segmentation.csv')

# Select relevant spending columns
X = df[spending_features]

# extract ratios for all the spending features
# Create ratio features for spending categories, excluding the Total_Spending itself
spending_categories = [feature for feature in spending_features if feature != 'Total_Spending']
for feature in spending_categories:
    X[f'{feature}_ratio'] = X[feature] / X['Total_Spending'].replace(0, 1)  # Avoid division by zero

print(X.head())

df_ratios= X[[f'{feature}_ratio' for feature in spending_categories]]


# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_ratios)

print(df_ratios.head())



3. Optimizing the **number of clusters** using the Elbow Method

In [ ]:
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

print(inertia)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Spending Pattern Clustering')
plt.grid(True)
plt.show()


4. **Applying KMeans Clustering** with the optimal number of clusters
We will choose 3 clusters based on the elbow method.
this is because the inertia starts to level off after 3 clusters, indicating that adding more clusters does not significantly reduce inertia.

In [ ]:
# Apply KMeans clustering with clusters
kmeans = KMeans(n_clusters=3, random_state=42)
df['Spending_Cluster'] = kmeans.fit_predict(X_scaled)


cluster_summary = df.groupby('Spending_Cluster')[spending_features].mean().round(2)
print(cluster_summary)


5. **Analyzing Cluster Characteristics**

We will analyze the characteristics of each cluster by computing the mean spending for each product category.

We will also visualize the average spending per product category for each cluster using a grouped bar chart.


In [ ]:
# Group by cluster and compute means
cluster_means = df.groupby('Spending_Cluster')[spending_features].mean()

print(cluster_means)

# Transpose for easier plotting (products on x-axis)
cluster_means_T = cluster_means.T

# Plot grouped bar chart
cluster_means_T.plot(kind='bar', figsize=(8, 6))

plt.title('Average Spending per Product Category by Cluster')
plt.xlabel('Product Category')
plt.ylabel('Average Spending')
plt.legend(title='Cluster')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

This is a stacked bar chart showing the average spending per product category for each cluster. Each segment of the bar represents the average spending for a specific product category, and different colors represent different clusters. This visualization helps us understand how each cluster allocates its spending across various product categories.

In [ ]:
# Normalize category spending by total spending per cluster
normalized = df.groupby('Spending_Cluster')[
    ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
     'MntSweetProducts', 'MntGoldProds']
].mean()

# Add total spending per cluster
normalized['Total'] = normalized.sum(axis=1)

# Divide each category by total to get proportions
normalized_pct = normalized.div(normalized['Total'], axis=0).drop(columns='Total')

# Optional: Multiply by 100 for percentage
normalized_pct *= 100

print(normalized_pct.round(2))

# plot the normalized dta
# Create a colorful stacked bar chart with improved styling
plt.figure(figsize=(12, 8))

# Use a colorful palette
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']
normalized_pct.plot(
    kind='bar',
    stacked=True,  # Use stacked bars to show composition
    figsize=(12, 8),
    color=colors,
    width=0.7,  # Slightly thinner bars
    edgecolor='white'  # White edge for contrast between segments
)

# Enhance styling
plt.title('Spending Composition by Customer Segment', fontsize=16, pad=20)
plt.xlabel('Customer Segment', fontsize=14, labelpad=10)
plt.ylabel('Percentage of Total Spending (%)', fontsize=14, labelpad=10)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)

# Add percentage signs to y-axis
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:.0f}%'.format(x) for x in current_values])

# Improve legend
plt.legend(
    title='Product Category',
    title_fontsize=12,
    fontsize=10,
    loc='upper right',
    bbox_to_anchor=(1.15, 1)
)

# Add value labels on the stacked bars
for i, cluster in enumerate(normalized_pct.index):
    cumulative = 0
    for j, col in enumerate(normalized_pct.columns):
        value = normalized_pct.loc[cluster, col]
        if value > 5:  # Only show labels for segments > 5%
            plt.text(
                i,
                cumulative + value / 2,  # Center of segment
                f"{value:.1f}%",
                ha='center',
                va='center',
                fontsize=9,
                fontweight='bold',
                color='black' if value > 15 else 'white'  # Better contrast
            )
        cumulative += value

# Add grid lines for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Rename x-axis labels to more meaningful names
plt.gca().set_xticklabels([
    'Segment 0',
    'Segment 1',
    'Segment 2'
])

plt.tight_layout()
plt.show()


In [ ]:
# average income per cluster
income_per_cluster = df.groupby('Spending_Cluster')['Income'].mean().round(2)
print("Average Income per Cluster:")
print(income_per_cluster)

# Plot average income per cluster
plt.figure(figsize=(8, 6))
income_per_cluster.plot(kind='bar', color='skyblue')
plt.title('Average Income per Customer Segment')
plt.xlabel('Customer Segment')
plt.ylabel('Average Income')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
#order of product preference per category
# Dictionary to store cluster-wise hierarchy
cluster_hierarchy = {}
cluster_means.drop('Total_Spending', axis=1, inplace=True)  # Drop Total_Spending for hierarchy
# Loop through each cluster and sort categories
for cluster_id in cluster_means.index:
    sorted_categories = cluster_means.loc[cluster_id].sort_values(ascending=False).index.tolist()
    cluster_hierarchy[cluster_id] = sorted_categories

# Display the result
for cluster, preferences in cluster_hierarchy.items():
    print(f"Cluster {cluster} preference hierarchy: {preferences}")


In [ ]:
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA



print(" SCATTER PLOT ANALYSIS FOR SPENDING CLUSTERS")
print("="*60)

# 1. PCA Scatter Plot - Most comprehensive view of all spending patterns
print("\n1. PCA-based Scatter Plot for Spending Cluster Visualization")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['Spending_Cluster'], 
                     cmap='viridis', alpha=0.7, s=60, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Spending Cluster')
plt.xlabel(f'First Principal Component (explains {pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component (explains {pca.explained_variance_ratio_[1]:.1%} variance)')
plt.title('Customer Spending Behavior Clusters (PCA View)', fontsize=16, pad=20)
plt.grid(True, alpha=0.3)

# Add cluster centers
centers_pca = pca.transform(kmeans.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', marker='x', 
           s=300, linewidths=4, label='Cluster Centers')
plt.legend()
plt.tight_layout()
plt.show()

print(f"PCA explains {sum(pca.explained_variance_ratio_):.1%} of total variance in spending patterns")